We'll start off by setting the seed for reproducibility.

In [1]:
import sys
import torch
import random
import numpy as np

seed = 26

In [2]:
import sys
if 'google.colab' in sys.modules:  # If in Google Colab environment
    # Installing requisite packages
    !pip install datasets transformers evaluate
    !pip install accelerate -U
    !pip install -U sentence-transformers

    # Mount google drive to enable access to data files
    from google.colab import drive
    drive.mount('/content/drive')

    # Change working directory to desired folder
    %cd /content/drive/MyDrive/HS_23_Msc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 87.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.0 MB/

# Reading data

In [3]:
from transformers import pipeline
import pandas as pd
import torch

In [4]:
# Reading in the .csv data
dat = pd.read_csv('Embeddings_text.csv')
dat # Inspecting the data

,word+comment
0,Klimagesetz
1,Teuer
2,Stromteuerung
3,Hohe Kosten Stromleitungen
4,PV privat Unsinn
...,...
6672,Verbot fossiler Energie Direkt alle fossile En...
6673,Unterstützung für Umstellung Wann wird unterst...
6674,Schutz für Generationen Wir schützen die zukün...
6675,Lösung von Ausland Wir müssen danach weniger i...


# Extracting features

In [5]:
# Setting device to mps
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [6]:
# Initializing feature extractor
model_ckpt = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
feature_extractor = pipeline(
    'feature-extraction', model=model_ckpt, tokenizer=model_ckpt, device=device,
    framework='pt'
)

# Extracting features
texts = dat['word+comment'].tolist()
features = [feature_extractor(text, return_tensors='pt')[0][0].numpy() for text in texts]
features = pd.DataFrame(features)
features

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
0,0.022616,0.415679,0.131571,0.320233,0.477956,0.060180,0.026229,-0.137214,-0.032047,0.342856,...,0.087627,-0.028879,0.130957,-0.066824,-0.387957,0.116138,-0.240112,-0.287597,-0.305492,0.033682
1,0.116735,0.307324,-0.068396,-0.248067,0.116704,-0.071267,0.342812,0.516092,0.175445,-0.018673,...,0.126176,-0.088379,0.028152,-0.058135,-0.285814,0.191686,0.137779,-0.065865,-0.063493,0.294646
2,-0.094757,0.562128,-0.044198,0.292374,0.324770,-0.193191,0.242892,0.277727,0.019026,0.093243,...,0.393377,-0.144331,0.088521,-0.083992,-0.406141,-0.099503,0.053925,0.101875,-0.094818,0.038419
3,-0.099935,0.261219,0.101228,0.162826,0.106694,-0.241507,0.143845,0.250619,0.034052,-0.024762,...,0.330932,-0.185408,-0.162031,-0.121693,-0.459234,-0.148149,0.038668,-0.021917,-0.079975,-0.328134
4,0.061350,0.271958,-0.092156,-0.227568,0.200132,-0.261179,-0.037341,0.346176,0.020389,0.113451,...,0.293944,-0.238364,-0.001133,0.073183,-0.340103,0.010322,0.184412,0.045912,-0.194272,0.166380
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6672,0.116078,0.658633,0.246164,0.093990,0.262883,-0.172630,-0.004747,0.081684,-0.084377,-0.005364,...,0.268384,0.088600,-0.136595,-0.150900,-0.602492,0.105352,0.406598,-0.417531,0.110852,-0.221163
6673,0.058566,0.229127,0.117543,0.017403,0.135865,-0.024075,0.087522,0.057874,-0.230994,0.127225,...,0.283298,-0.155944,0.011415,0.167112,-0.038907,0.023881,0.156784,0.039738,-0.133683,0.025509
6674,0.025202,0.543096,-0.534469,-0.276423,0.246454,0.320457,0.322848,0.122149,0.150811,0.341115,...,0.258940,0.176482,-0.053111,0.057186,-0.210958,0.241228,0.193425,-0.322262,0.074440,0.176749
6675,0.122912,0.080157,0.133012,-0.140703,0.180823,-0.058275,0.121257,0.011021,-0.070273,0.205948,...,0.118100,-0.017532,-0.047305,0.022218,-0.140578,0.059734,0.022427,-0.018349,0.262226,-0.070801


In [7]:
features.to_csv('features2.csv')